In [1]:
%time from data import *

CPU times: user 12.9 s, sys: 1.42 s, total: 14.3 s
Wall time: 14.8 s


# pair exploration

In [6]:
item_pairs_train.isDuplicate.value_counts(normalize=True)

0    0.577362
1    0.422638
Name: isDuplicate, dtype: float64

In [22]:
item_pairs_train.generationMethod.value_counts(normalize=True)

1    0.682492
3    0.270067
2    0.047441
Name: generationMethod, dtype: float64

In [5]:
item_pairs_train.groupby('generationMethod')['isDuplicate'].mean()

generationMethod
1    0.508611
2    1.000000
3    0.103955
Name: isDuplicate, dtype: float64

duplication ratio is quite diffent acrosss generationMethod

In [14]:
(item_pairs_train.itemID_1 < item_pairs_train.itemID_2).all()

True

In [23]:
(item_pairs_test.itemID_1 < item_pairs_test.itemID_2).all()

True

itemID_1 < itemID_2

In [10]:
item_pairs_train.head()

,itemID_1,itemID_2,isDuplicate,generationMethod
0,1,4112648,1,1
1,3,1991275,1,1
2,4,1223296,0,1
3,7,1058851,1,1
4,8,2161930,1,1


In [12]:
item_pairs_train.groupby(['itemID_1', 'itemID_2'])['isDuplicate'].count().max()

1

In [24]:
item_pairs_test.groupby(['itemID_1', 'itemID_2'])['itemID_1'].count().max()

1

item pair is unique

In [29]:
ids_train = set(item_pairs_train.itemID_1.tolist() + item_pairs_train.itemID_2.tolist())
n_ids_train = len(ids_train)
print(n_ids_train)

3344613


In [30]:
item_pairs_train.shape[0] / ((n_ids_train**2 -n_ids_train)/2)

5.348257232893055e-07

In [31]:
ids_test = set(item_pairs_test.itemID_1.tolist() + item_pairs_test.itemID_2.tolist())
n_ids_test = len(ids_test)
print(n_ids_test)

1315205


In [32]:
item_pairs_test.shape[0] / ((n_ids_test**2 -n_ids_test)/2)

1.2073284887691106e-06

* only tiny fraction is labelled
* test pair is more than train

In [33]:
ids_train.intersection(ids_test)

set()

In [38]:
sorted(ids_train)[:20]

[1, 3, 4, 7, 8, 9, 12, 15, 16, 19, 20, 21, 22, 26, 28, 29, 32, 33, 35, 39]

In [42]:
sorted(ids_test)[:10]

[5, 6, 11, 23, 30, 31, 36, 47, 48, 52]

In [44]:
len(ids_train) + len(ids_test) - max(max(ids_train), max(ids_test))

-1452185

* train, test id has no intersection
* it's not complete as id